# XFEL Wave Optics Simulation Tutorial - 1

L.Samoylova <liubov.samoylova@xfel.eu>, A.Buzmakov <buzmakov@gmail.com>

Tutorial course on FEL Wavefront Propagation Simulations, 05/10/2016, SOS2016 Workshop, Trieste.

Version 14/10/2016

Wave optics software is based on SRW core library <https://github.com/ochubar/SRW>, available through WPG interactive framework <https://github.com/samoylv/WPG>

## FEL Source Simulation

### Import modules

In [ ]:
%matplotlib inline

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals


#Importing necessary modules:
import os
import sys
import copy
import time
import errno
import numpy as np
import pylab as plt

from IPython.display import Image

### change `wpg_path` to root folder of the WPG installation

In [ ]:
# wpg_path = '/Users/lsamoylv/code/ghub/latest/WPG/'
#wpg_path = '/Users/lsamoylv/code/ghub/srw_update/'
wpg_path = os.path.join('..','WPG')
sys.path.insert(0,wpg_path)

#Import base wavefront class
from wpg import Wavefront

#Gaussian beam generator
from wpg.generators import build_gauss_wavefront

#import some helpers functions
from wpg.wpg_uti_exfl import calculate_theta_fwhm_cdr_s1
from wpg.wpg_uti_wf import calc_pulse_energy, averaged_intensity, calculate_fwhm, get_intensity_on_axis
from wpg.wpg_uti_wf import plot_wf, look_at_q_space

plt.ion()

### define helper functions

In [ ]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

### define source parameters

In [ ]:
data_path = 'data_example_01';mkdir_p(data_path)
out_prefix = 'gb_'
src_oe1 = 257.8 # Distance to the first OE [m]

# Central photon energy.
ekev = 8.5 # Energy [keV]

# Pulse parameters.
qnC = 0.1               # e-bunch charge, [nC]
pulse_duration = 9.e-15 # [s] <-is not used really, only ~coh time pulse duration has physical meaning 
pulseEnergy = 1.5e-3    # total pulse energy, J
coh_time = 0.8e-15      # [s]<-should be SASE coherence time, then spectrum will be the same as for SASE 
                        # check coherence time for 8 keV 0.5 nC SASE1

# Angular distribution
theta_fwhm = calculate_theta_fwhm_cdr_s1(ekev,qnC) # CDR2011 empirical formula
theta_fwhm = 3.2e-6 #empiric value for nzc 15 (linear SASE before saturation)
print('divergence FWHM {0:.1f} urad'.format(theta_fwhm*1e6))

### Gaussian beam

#### Phase delay $\phi(r)=\frac{kr^2}{2R}$ 
<img src='figures/phase_delay.png'>


In [ ]:
Image(filename='figures/gaussian_beam.png')

### define Gaussian beam parameters

In [ ]:
# Gaussian beam parameters
wlambda = 12.4*1e-10/ekev        # wavelength 
w0 = wlambda/(np.pi*theta_fwhm)  # FWHM beam waist; 
w0_a = w0/(2*np.sqrt(np.log(2))) # sigma of amplitude
zR = (np.pi*w0_a**2)/wlambda     # Rayleigh length, max WF curvature

# expected beam radius at OE1 position to get the range of the wavefront 
print('Intensity waist size: {:.0f} um FWHM'.format(w0*1e6))
print('Rayleigh length: {:.1f} m'.format(zR))
print('expected FWHM at distance {:.1f} m: {:.2f} mm'.format(src_oe1,theta_fwhm*src_oe1*1e3))

fname = '{0:s}at_{1:.0f}_m'.format(out_prefix,src_oe1)

In [ ]:
bSaved=True
sig_num = 5.5
num_points = 400 #number of points
range_xy = w0_a * np.sqrt(1+(src_oe1/zR)**2)/np.sqrt(2)*sig_num;
print('range_xy at OE1: {:.1f} mm'.format(range_xy*1e3))
dx = 10.e-6; range_xy = dx*(num_points-1);
print('range_xy at OE1: {:.1f} mm'.format(range_xy*1e3))
nslices = 20; 

### create Gaussian beam

In [ ]:
srwl_wf = build_gauss_wavefront(num_points, num_points, nslices, ekev, -range_xy/2, range_xy/2,
                                -range_xy/2, range_xy/2 ,coh_time/np.sqrt(2), 
                                w0_a, w0_a, src_oe1,
                                pulseEn=pulseEnergy, pulseRange=8.)
wf = Wavefront(srwl_wf)
z0 = src_oe1
#defining name HDF5 file for storing wavefront
#store wavefront to HDF5 file 
if bSaved:     
    wf.store_hdf5(os.path.join(data_path,fname+'.h5')); 
    print('saving WF to {:s}'.format(os.path.join(data_path,fname+'.h5')))

xx=calculate_fwhm(wf);
print('FWHM at distance {:.1f} m: {:.2f} x {:.2f} mm2'.format(z0,xx[u'fwhm_x']*1e3,xx[u'fwhm_y']*1e3));

### show Gaussian wavefront in R- and Q-space

In [ ]:
plot_wf(wf)
look_at_q_space(wf)

### load FEL SASE pulse from [XFEL Photon Database](https://in.xfel.eu/xpd/)

In [ ]:
data_path = 'data_example_01';mkdir_p(data_path)
fname_prefix = 's1_8_43_100_xy75_ss14_nzc15'  # wavefront file
fname = fname_prefix+'_1_0000001'
full_fname = os.path.join(data_path,fname+'.h5')  # wavefront file
request = '57f354b36aa81'

In [ ]:
if os.path.exists(full_fname):
    print('Data file already exist. Skip downloading. File name: {}'.format(full_fname))
else:
    #zip_file =  os.path.join(data_path,'s1_8_43_100_xy75_ss10_nzc15_1_0000001.h5.zip') # downloaded zip file 
    #zip_file =  full_fname+'.zip') # downloaded zip file 
    zip_file =  fname_prefix+"_hdf5.zip"
    if not os.path.exists(os.path.join(data_path,zip_file)):
        print('Data file not found. File name: {}'.format(zip_file))
        print('Please wait... I\'m downloading. File size should be 239.31 MB')
        
        try: 
            from urllib import urlretrieve 
        except ImportError:
            from urllib.request import urlretrieve
            
        urlretrieve(  #  download path
            "https://in.xfel.eu/xpd/output/2016/10/"+request+"/"+zip_file, 
            os.path.join(data_path,zip_file)) 
        print('Download completed.')
    

    
    print('Unziping...')
    import zipfile
    zf= zipfile.ZipFile(os.path.join(data_path,zip_file))
    zf.extractall(data_path)
    print('Unzip file finished.')
        

### load and show initial FEL SASE wavefront

In [ ]:
from wpg.srwlib import srwl
wf=Wavefront()
wf.load_hdf5(full_fname)
#Resizing: decreasing Range of Horizontal and Vertical Position:
#srwl.ResizeElecField(wf._srwl_wf, 'c', [0, 0.48, 0.48/0.16, 0.48,  0.48/0.16]);
fwhm=calculate_fwhm(wf);
print('source size FWHM: {:.3f} x {:.3f} mm2'.format(fwhm[u'fwhm_x']*1e3,fwhm[u'fwhm_y']*1e3));
plot_wf(wf)
srwl.SetRepresElecField(wf._srwl_wf, 'f')
print('resizing in frequency domain...');t0 = time.time();
srwl.ResizeElecField(wf._srwl_wf, 'f', [0, 0.25, 0.25/0.25]);plot_wf(wf)
print('done in {:.2f} s'.format(time.time()-t0))


### propagate SASE FEL pulse by 1st optical element aperture¶

In [ ]:
from wpg import Beamline
from wpg.optical_elements import Drift, Use_PP
bl = Beamline()
z = src_oe1 - zR
zoom = 1.5
bl.append(Drift(z), Use_PP(
        semi_analytical_treatment=1, zoom=zoom, sampling=zoom/1.2))
print(bl)
wf=Wavefront()
print('loading WF from the file {}..'.format(full_fname));t0 = time.time();
wf.load_hdf5(full_fname)
print('done in {:.2f} s'.format(time.time()-t0))
print('switching to frequency domain..');t0 = time.time();
srwl.SetRepresElecField(wf._srwl_wf, 'f') # <---- switch to frequency domain
print('done in {:.2f} s'.format(time.time()-t0))
print('propagating..');t0 = time.time();
bl.propagate(wf)
print('done in {:.2f} s'.format(time.time()-t0))
print('switching to frequency domain..');t0 = time.time();
srwl.SetRepresElecField(wf._srwl_wf, 't')
print('done in {:.2f} s'.format(time.time()-t0))
fwhm=calculate_fwhm(wf);
wf.custom_fields['/params/beamline/printout'] = str(bl)
print('FWHM at distance {:.1f} m: {:.2f} x {:.2f} mm2'.format(z,fwhm[u'fwhm_x']*1e3,fwhm[u'fwhm_y']*1e3));

### show propagated FEL SASE wavefront

In [ ]:
plot_wf(wf)
look_at_q_space(wf)    

### save propagated pulse 

In [ ]:
fname_out = '{}_prop_at_{:03d}m'.format(fname,int(z))
z = src_oe1 - zR
if bSaved:     
    wf.store_hdf5(os.path.join(data_path,fname_out+'.h5')); 
    print('saving WF to {:s}'.format(os.path.join(data_path,fname_out+'.h5')))